In [1]:
import cv2
import mediapipe as mp
import numpy as np
import math

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle (a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End

    rad = np.abs(math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1],a[0]-b[0]))
    angle=(np.abs(rad* 180.0/np.pi))
    #return angle
    return angle

In [2]:
def kalman_angle():
    kalman  = cv2.KalmanFilter(4,1)#(dynamparams,measureparams)
    delta_t = 0.3
    c = 1
# measurementMatrix H 
    kalman.measurementMatrix = np.array([[1,0,0,0]],np.float32)
    print("kalman")
# transitionMatrix F 
    kalman.transitionMatrix = np.array([[1,delta_t,0.5*delta_t**2,(1/6)*delta_t**3],[0,1,delta_t,0.5*delta_t**2],[0,0,1,delta_t],[0,0,0,1]],np.float32)# processNoiseCov Q 
# processNoiseCov Q
    kalman.processNoiseCov = np.array([[(1/36)*delta_t**6,(1/12)*delta_t**5,(1/6)*delta_t**4,(1/6)*delta_t**3],[(1/12)*delta_t**5,(1/4)*delta_t**4,(1/2)*delta_t**3,(1/2)*delta_t**2],[(1/6)*delta_t**4,0.5*delta_t**3,delta_t**2,delta_t],[(1/6)*delta_t,0.5*delta_t**2,delta_t,1]],np.float32) * c**2
    return kalman

In [3]:
def video():
    cap =cv2.VideoCapture(0)
    kalman = kalman_angle()
# Curl counter variables
    counter=0
    stage = None
    new_stage = None
    new_counter=0
   # current_measurement = np.array((2,1),np.float32)
    current_prediction = np.zeros((2,1),np.float32)
    
    # Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            #global current_measurement,measurements,last_measurement,current_prediction,last_prediction
            
            ret,frame = cap.read()
        
        # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
        #Make detection
            results = pose.process(image)
        
        # Recolor image to BGR
            image.flags.writeable =True
            image= cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # print (results)
        # cv2.imshow("Mediapipe Feed", frame)
        
        # Extrack landmarks
            
            if(results.pose_landmarks==None):
                continue
            landmarks = results.pose_landmarks.landmark
                
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            #print (landmarks)
            
            # Calculate angle
            angle=calculate_angle(shoulder,elbow,wrist)
                
            current_measurement = np.array(np.float32(angle))
                
            kalman.correct(current_measurement)
            print ('measurement_angle: ',angle)
            current_prediction = kalman.predict()                
            new_angle = float(current_prediction[0])
            print('prediction_angle: ',new_angle)
                #current_measurement = np.array([angle])
              #  kalman.correct(current_measurement)
              #  current_prediction = kalman.predict()
                
            # Curl counter logic
            if angle > 160:
                stage ="down"
            if angle < 45 and stage == 'down':
                stage ="up"
                counter +=1
               # print (counter)
        # Curl counter logic
            if new_angle > 160:
                new_stage ="down"
            if new_angle < 45 and new_stage == 'down':
                new_stage ="up"
                new_counter +=1
               # print (counter)
            
     # Visualize angle
            
            #position=tuple(np.multiply(elbow, [640,480]).astype(int))
            position=(100, 100)
            font =cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(image, str(counter),position,font,2,(255,255,255),2,cv2.LINE_AA)
            
            new_position=(100, 200)
            new_font =cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(image, str(new_counter),new_position,new_font,2,(0,100,0),2,cv2.LINE_AA)
                                      
            #print (landmarks)
            
        
            
        # Render detections
            mp_drawing.draw_landmarks (image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  )
            cv2.imshow("Mediapipe Feed", image)

            if cv2.waitKey(10) & 0xFF == ord ('q'):
                break
        
    cap.release()
    cv2.destroyAllWindows() 
    cv2.waitKey(1)       

In [4]:
video()

kalman


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


measurement_angle:  1.6411613639555898
prediction_angle:  0.0
measurement_angle:  1.4232762386824263
prediction_angle:  0.00023056607460603118
measurement_angle:  1.5397349186842866
prediction_angle:  0.008065345697104931
measurement_angle:  1.9806387906138048
prediction_angle:  0.09900524467229843
measurement_angle:  2.9744393203321615
prediction_angle:  0.7286025285720825
measurement_angle:  3.93952791849559
prediction_angle:  2.868488311767578
measurement_angle:  4.328193445796191
prediction_angle:  5.791109561920166
measurement_angle:  4.1412440872116925
prediction_angle:  7.328878879547119
measurement_angle:  4.07344273367894
prediction_angle:  7.589675426483154
measurement_angle:  4.047505875104962
prediction_angle:  7.095417499542236
measurement_angle:  3.926455811539824
prediction_angle:  6.034820079803467
measurement_angle:  4.407134998371403
prediction_angle:  5.30138635635376
measurement_angle:  3.77051216082211
prediction_angle:  3.783550977706909
measurement_angle:  4.3161

In [7]:
def video():
    cap =cv2.VideoCapture(0)
    kalman = kalman_angle()
# Curl counter variables
    counter=0
    stage = None
    new_stage = None
    new_counter=0
    last_measurement = current_measurement = np.array((2,1),np.float32)
    last_predicition = current_prediction = np.zeros((2,1),np.float32)
    
    # Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            #global current_measurement,measurements,last_measurement,current_prediction,last_prediction
            
            ret,frame = cap.read()
        
        # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
        #Make detection
            results = pose.process(image)
        
        # Recolor image to BGR
            image.flags.writeable =True
            image= cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # print (results)
        # cv2.imshow("Mediapipe Feed", frame)
        
        # Extrack landmarks
            
        
            try:
                landmarks = results.pose_landmarks.landmark
                
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            #print (landmarks)
            
            # Calculate angle
                angle=calculate_angle(shoulder,elbow,wrist)
            
            
                last_measurement = current_measurement
                last_prediction = current_prediction
                
                current_measurement = np.array(np.float32(angle))
                
                kalman.correct(current_measurement)
                print ('measurement_angle: ',angle)
                current_prediction = kalman.predict()                
                new_angle = float(current_prediction[0])
                print('prediction_angle: ',new_angle)
                #current_measurement = np.array([angle])
              #  kalman.correct(current_measurement)
              #  current_prediction = kalman.predict()
                
            # Curl counter logic
                if angle > 160:
                    stage ="down"
                if angle < 30 and stage == 'down':
                    stage ="up"
                    counter +=1
               # print (counter)
        # Curl counter logic
                if new_angle > 160:
                    new_stage ="down"
                if new_angle < 30 and new_stage == 'down':
                    new_stage ="up"
                    new_counter +=1
               # print (counter)
            
     # Visualize angle
            
            #position=tuple(np.multiply(elbow, [640,480]).astype(int))
                position=(100, 100)
                font =cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(image, str(counter),position,font,2,(255,255,255),2,cv2.LINE_AA)
            
                new_position=(100, 200)
                new_font =cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(image, str(new_counter),new_position,new_font,2,(0,100,0),2,cv2.LINE_AA)
                                      
            #print (landmarks)
            
            except:
                pass
        # Render detections
            mp_drawing.draw_landmarks (image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  )
            cv2.imshow("Mediapipe Feed", image)

            if cv2.waitKey(10) & 0xFF == ord ('q'):
                break
        
    cap.release()
    cv2.destroyAllWindows() 
    cv2.waitKey(1)       